In [ ]:
import pandas as pd
import kagglehub

# Download the dataset
path = kagglehub.dataset_download("sid321axn/beijing-multisite-airquality-data-set")
print("Path to dataset files:", path)

# File names to be processed
list_filenames = [
    "PRSA_Data_Guanyuan_20130301-20170228.csv",
    "PRSA_Data_Aotizhongxin_20130301-20170228.csv",
    "PRSA_Data_Wanliu_20130301-20170228.csv",
    "PRSA_Data_Tiantan_20130301-20170228.csv",
    "PRSA_Data_Wanshouxigong_20130301-20170228.csv",
    "PRSA_Data_Nongzhanguan_20130301-20170228.csv",
    "PRSA_Data_Shunyi_20130301-20170228.csv",
    "PRSA_Data_Changping_20130301-20170228.csv",
    "PRSA_Data_Dingling_20130301-20170228.csv",
    "PRSA_Data_Huairou_20130301-20170228.csv",
    "PRSA_Data_Gucheng_20130301-20170228.csv",
    "PRSA_Data_Dongsi_20130301-20170228.csv"
]

# Load datasets into a list of dataframes
dataframes = []
for filename in list_filenames:
    full_path = f"{path}/{filename}"
    df = pd.read_csv(full_path)
    df['site'] = filename.split('_')[2]  # Add site identifier
    dataframes.append(df)

# Combine all dataframes into one
combined_data = pd.concat(dataframes, ignore_index=True)

# Display combined data structure
print("Combined Dataset Info:")
print(combined_data.info())

Path to dataset files: /root/.cache/kagglehub/datasets/sid321axn/beijing-multisite-airquality-data-set/versions/1
Combined Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420768 entries, 0 to 420767
Data columns (total 19 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   No       420768 non-null  int64  
 1   year     420768 non-null  int64  
 2   month    420768 non-null  int64  
 3   day      420768 non-null  int64  
 4   hour     420768 non-null  int64  
 5   PM2.5    412029 non-null  float64
 6   PM10     414319 non-null  float64
 7   SO2      411747 non-null  float64
 8   NO2      408652 non-null  float64
 9   CO       400067 non-null  float64
 10  O3       407491 non-null  float64
 11  TEMP     420370 non-null  float64
 12  PRES     420375 non-null  float64
 13  DEWP     420365 non-null  float64
 14  RAIN     420378 non-null  float64
 15  wd       418946 non-null  object 
 16  WSPM     420450 non-null  float64
 17  statio

In [ ]:
import pandas as pd
import numpy as np

# Create a datetime column
combined_data['datetime'] = pd.to_datetime(combined_data[['year', 'month', 'day', 'hour']])

# Sort the dataset by datetime
combined_data = combined_data.sort_values(by='datetime')

# Verify the update
print("Data sorted by datetime successfully.")
print(combined_data[['datetime', 'year', 'month', 'day', 'hour']].head())

# Sort data by time for temporal consistency
combined_data = combined_data.sort_values(by='datetime')

# Inspect missing values
missing_summary = combined_data.isnull().sum()

# Drop columns with too many missing values or irrelevant ones
threshold = 0.5 * len(combined_data)  # Drop columns with >50% missing
combined_data_cleaned = combined_data.dropna(axis=1, thresh=threshold)

# Fill remaining missing values with forward fill or a constant
combined_data_cleaned = combined_data_cleaned.fillna(method='ffill').fillna(0)

# Feature Engineering
# Add temporal features
combined_data_cleaned['hour'] = combined_data_cleaned['datetime'].dt.hour
combined_data_cleaned['day'] = combined_data_cleaned['datetime'].dt.day
combined_data_cleaned['month'] = combined_data_cleaned['datetime'].dt.month
combined_data_cleaned['year'] = combined_data_cleaned['datetime'].dt.year
combined_data_cleaned['day_of_week'] = combined_data_cleaned['datetime'].dt.dayofweek
combined_data_cleaned['is_weekend'] = combined_data_cleaned['day_of_week'].isin([5, 6]).astype(int)

# Add a target variable for 7-day ahead prediction
combined_data_cleaned['target_7_days_ahead'] = combined_data_cleaned['PM2.5'].shift(-7)  # Shift PM2.5 by 7 days

# Add spatial granularity
combined_data_cleaned['city_id'] = combined_data_cleaned['site'].astype('category').cat.codes

# Generate lag features for AQI or other pollutants
lag_features = ['PM2.5', 'PM10', 'O3']
for feature in lag_features:
    for lag in range(1, 8):  # Create lag features for up to 7 days
        combined_data_cleaned[f'{feature}_lag_{lag}'] = combined_data_cleaned[feature].shift(lag)

# Generate rolling statistics
for feature in lag_features:
    combined_data_cleaned[f'{feature}_rolling_mean'] = combined_data_cleaned[feature].rolling(window=7).mean()
    combined_data_cleaned[f'{feature}_rolling_std'] = combined_data_cleaned[feature].rolling(window=7).std()

# Drop rows with NaN values introduced by rolling or lagging
combined_data_cleaned = combined_data_cleaned.dropna()

# Save cleaned and feature-engineered data to a CSV for further use
combined_data_cleaned.to_csv('processed_beijing_airquality_data.csv', index=False)

# Output dataset structure and head
print("Processed Dataset Info:")
print(combined_data_cleaned.info())
print(combined_data_cleaned.head())


Data sorted by datetime successfully.
         datetime  year  month  day  hour
0      2013-03-01  2013      3    1     0
315576 2013-03-01  2013      3    1     0
70128  2013-03-01  2013      3    1     0
350640 2013-03-01  2013      3    1     0
35064  2013-03-01  2013      3    1     0


<ipython-input-4-f1ecf49959b3>:25: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_data_cleaned = combined_data_cleaned.fillna(method='ffill').fillna(0)


Processed Dataset Info:
<class 'pandas.core.frame.DataFrame'>
Index: 420754 entries, 35064 to 175319
Data columns (total 51 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   No                   420754 non-null  int64         
 1   year                 420754 non-null  int32         
 2   month                420754 non-null  int32         
 3   day                  420754 non-null  int32         
 4   hour                 420754 non-null  int32         
 5   PM2.5                420754 non-null  float64       
 6   PM10                 420754 non-null  float64       
 7   SO2                  420754 non-null  float64       
 8   NO2                  420754 non-null  float64       
 9   CO                   420754 non-null  float64       
 10  O3                   420754 non-null  float64       
 11  TEMP                 420754 non-null  float64       
 12  PRES                 420754 non-null  float64    

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt
from sklearn.preprocessing import StandardScaler


# Identify numeric columns
numeric_columns = combined_data_cleaned.select_dtypes(include=[np.number]).columns

# Step 1: Prepare the data for LSTM with new features
def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length, :-1])  # features
        y.append(data[i+sequence_length, -1])    # target variable (PM2.5)
    return np.array(X), np.array(y)

sequence_length = 7  # Use the last 7 days to predict the next day
X, y = create_sequences(combined_data_cleaned[numeric_columns].values, sequence_length)

# Split data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Normalize the data
scaler = StandardScaler()

# X_train and X_test are 3D arrays (samples, time steps, features) for LSTM, scaling should be applied per feature
X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])  # Flatten the time steps for scaling
X_test_reshaped = X_test.reshape(-1, X_test.shape[-1])

X_train_scaled = scaler.fit_transform(X_train_reshaped).reshape(X_train.shape)  # Reshape back to 3D
X_test_scaled = scaler.transform(X_test_reshaped).reshape(X_test.shape)  # Reshape back to 3D

# Step 2: Build the LSTM Model (for hyperparameter tuning)
def build_lstm_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=128, step=32),
                   return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(0.2))
    model.add(Dense(1))  # Output the prediction (PM2.5 value)
    model.compile(optimizer='adam', loss='mse')
    return model

# Initialize Keras Tuner for LSTM tuning
tuner = kt.Hyperband(build_lstm_model, objective='val_loss', max_epochs=10, factor=3, directory='my_dir', project_name='lstm_tuning')

# Perform the tuning
tuner.search(X_train_scaled, y_train, epochs=10, validation_data=(X_test_scaled, y_test))

# Best hyperparameters for LSTM
best_lstm_hp = tuner.get_best_hyperparameters()[0]
print("Best LSTM hyperparameters:", best_lstm_hp.values)

# Step 3: Train the Best LSTM Model with Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)

lstm_model = build_lstm_model(best_lstm_hp)
lstm_model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[early_stopping])

# Step 4: Get LSTM Predictions  to use as features for XGBoost
lstm_preds_train = lstm_model.predict(X_train_scaled)
lstm_preds_test = lstm_model.predict(X_test_scaled)

# Save LSTM predictions for next part (XGBoost)
np.save('lstm_preds_train.npy', lstm_preds_train)
np.save('lstm_preds_test.npy', lstm_preds_test)

# save the model
lstm_model.save('best_lstm_model.h5')


Trial 4 Complete [00h 05m 48s]
val_loss: 15.791834831237793

Best val_loss So Far: 15.791834831237793
Total elapsed time: 00h 14m 34s
Best LSTM hyperparameters: {'units': 128, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}
Epoch 1/20
10519/10519 ━━━━━━━━━━━━━━━━━━━━ 144s 14ms/step - loss: 79.4391 - val_loss: 23.0271
Epoch 2/20
10519/10519 ━━━━━━━━━━━━━━━━━━━━ 148s 14ms/step - loss: 35.2282 - val_loss: 14.8224
Epoch 3/20
10519/10519 ━━━━━━━━━━━━━━━━━━━━ 153s 15ms/step - loss: 26.7272 - val_loss: 13.1979
Epoch 4/20
10519/10519 ━━━━━━━━━━━━━━━━━━━━ 153s 15ms/step - loss: 17.7216 - val_loss: 12.5842
Epoch 5/20
10519/10519 ━━━━━━━━━━━━━━━━━━━━ 192s 14ms/step - loss: 15.4993 - val_loss: 13.2791
Epoch 6/20
10519/10519 ━━━━━━━━━━━━━━━━━━━━ 207s 14ms/step - loss: 13.1233 - val_loss: 11.9291
Epoch 7/20
10519/10519 ━━━━━━━━━━━━━━━━━━━━ 146s 14ms/step - loss: 14.4676 - val_loss: 11.6900
Epoch 8/20
10519/10519 ━━━━━━━━━━━━━━━━━━━━ 204s 14ms/step - loss: 12.9713 -

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import keras_tuner as kt

# Load the saved LSTM predictions
lstm_preds_train = np.load('lstm_preds_train.npy')
lstm_preds_test = np.load('lstm_preds_test.npy')


# Flatten the predictions to 1D arrays
lstm_preds_train = lstm_preds_train.flatten()
lstm_preds_test = lstm_preds_test.flatten()

# Concatenate the predictions
lstm_preds = np.concatenate([lstm_preds_train, lstm_preds_test])

# If the lengths don't match, extend lstm_preds with NaN values to match the original data length
if len(lstm_preds) < len(combined_data_cleaned):
    lstm_preds = np.pad(lstm_preds, (0, len(combined_data_cleaned) - len(lstm_preds)), constant_values=np.nan)

# Add LSTM predictions as a feature to the dataset
combined_data_cleaned['lstm_preds'] = lstm_preds

# Step 5: Prepare Data for XGBoost
X = combined_data_cleaned[['PM2.5_lag_1', 'PM10_lag_1', 'O3_lag_1', 'PM2.5_rolling_mean', 'PM10_rolling_mean', 'O3_rolling_mean', 'lstm_preds']]
y = combined_data_cleaned['target_7_days_ahead']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize data for XGBoost
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# y_train is sliced to match the size of X_train_scaled
y_train = y_train[:X_train_scaled.shape[0]]

# Slice X_train_scaled to match the size of lstm_preds_train
X_train_scaled = X_train_scaled[:len(lstm_preds_train)]

# Combine LSTM predictions with other features for XGBoost (train set)
X_train_xgb = np.column_stack([lstm_preds_train[:X_train_scaled.shape[0]], X_train_scaled])

# Adjust the length of lstm_preds_test to match X_test_scaled
if len(lstm_preds_test) > X_test_scaled.shape[0]:
    lstm_preds_test = lstm_preds_test[:X_test_scaled.shape[0]]
elif len(lstm_preds_test) < X_test_scaled.shape[0]:
    X_test_scaled = X_test_scaled[:len(lstm_preds_test)]

# Combine LSTM predictions with other features for XGBoost
X_test_xgb = np.column_stack([lstm_preds_test, X_test_scaled])

# Step 6: Hyperparameter Tuning for XGBoost using RandomizedSearchCV
xgb_model = xgb.XGBRegressor(objective='reg:squarederror')

# Define a very reduced hyperparameter grid for XGBoost
param_grid_xgb = {
    'n_estimators': [100],
    'learning_rate': [0.1],
    'max_depth': [6],
    'subsample': [1.0],
}


# Align lengths of X_train_xgb and y_train
min_length = min(X_train_xgb.shape[0], y_train.shape[0])
X_train_xgb = X_train_xgb[:min_length]
y_train = y_train[:min_length]

# Hyperparameter tuning with RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=xgb_model,
                                   param_distributions=param_grid_xgb,
                                   n_iter=3,
                                   scoring='neg_mean_squared_error',
                                   cv=2)
random_search.fit(X_train_xgb, y_train)

# Output the best hyperparameters
print("Best parameters for XGBoost:", random_search.best_params_)

# Predict using the best model
y_pred_xgb = random_search.best_estimator_.predict(X_test_xgb)

# Align lengths
if len(y_test) > len(y_pred_xgb):
    y_test = y_test[:len(y_pred_xgb)]

# Calculate metrics
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)

print(f"Hybrid Model (LSTM + XGBoost) RMSE: {rmse_xgb}")
print(f"Hybrid Model (LSTM + XGBoost) MAE: {mae_xgb}")

Length of lstm_preds_train: 336597
Length of lstm_preds_test: 84150
Length of combined_data_cleaned: 420754
Shape of X_train_xgb: (336597, 8)
Shape of y_train: (336603,)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 1 is smaller than n_iter=3. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters for XGBoost: {'subsample': 1.0, 'n_estimators': 100, 'max_depth': 6, 'learning_rate': 0.1}
Hybrid Model (LSTM + XGBoost) RMSE: 30.26101538855598
Hybrid Model (LSTM + XGBoost) MAE: 17.394490716805894
